## Android and ios app analysis(Dataquest project 1)

In this project, we are going to analyse the performance of applications from android and ios devices. 

This anaysis will demonstrate the performance of the free applications and the profit it generates as we are analyzing only the free apps. Since, we are analysing only the free apps, the profit is determined by the in-app advertisement and thus we will also get the user flow. 

This analysis will help app developer decide the type of free app they should focus on building to make maximum profit.

First we will do the data cleaning. Following are the data cleaning task we will perform:
- Removing inaccurate data
- Removing duplicate app entries
- Removing non-English apps
- Isolating the free apps

In [1]:
def explore_data(dataset,start,end,rows_and_columns=False):
    dataset_slice = dataset[start:end]
    for row in dataset_slice:
        print(row)
        print('\n')
    if rows_and_columns:
        print('Number of rows ',len(dataset))
        print('Number of columns ',len(dataset[0]))

In [2]:
from csv import reader

# Extracting the ios app data as a nested list
file1 = 'AppleStore.csv'
opened_file1 = open(file1)
read_file1 = reader(opened_file1)
ios_data = list(read_file1)

# Extrating the android data as a nested list
file2 = 'googleplaystore.csv'
opened_file2 = open(file2)
read_file2 = reader(opened_file2)
android_data = list(read_file2)

In [3]:

# exploring the dataset
# explore_ios = explore_data(ios_data,0,1,True)
# explore_android = explore_data(android_data,10471,10473,True)

The dataset is obtained from the following link:

[Applestore app dataset](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps)

[googleplaystore app dataset](https://www.kaggle.com/lava18/google-play-store-apps/)

As per the discussion, i tried to check the data error mentioned and found the error in 10473 instead of 10472 as i consider header as the first data row.


In [4]:
header = explore_data(android_data,0,1,True)
row_with_error = explore_data(android_data,10472,10474,True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


Number of rows  10842
Number of columns  13
['Xposed Wi-Fi-Pwd', 'PERSONALIZATION', '3.5', '1042', '404k', '100,000+', 'Free', '0', 'Everyone', 'Personalization', 'August 5, 2014', '3.0.0', '4.0.3 and up']


['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


Number of rows  10842
Number of columns  13


In [5]:
# I am deleting the incomplete row 10473
print(len(android_data))
del android_data[10473]
print(len(android_data))

10842
10841


In [6]:
unique_apps = []
duplicate_apps = []
for data in android_data:
    if data[0] in unique_apps:
        duplicate_apps.append(data[0])
    else:
        unique_apps.append(data[0])
print('Number of duplicate apps:',len(duplicate_apps))
print('Few apps that have duplicates',duplicate_apps[:5])

Number of duplicate apps: 1181
Few apps that have duplicates ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings']


From above, we noticed that we have lots of duplicate data for the same app. We need to remove the duplicate data. To remove the duplicate data, we are considering the "review count" as a filtering criterion. Since data is  more reliable and accurate if it has high number of reviewer, this criterion will be effecient one. 

Next, we are going to make a dictionary that stores the name of app as its key and the maximum reviews (if there exist duplicate) as its value.

In [7]:
reviews_max = {}

for data in android_data[1:]:
    name = data[0]
    n_reviews = float(data[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
        
    if name not in reviews_max:
        reviews_max[name] = n_reviews
        
print(len(reviews_max))

9659


As we have now formed the dictionary with unique name, we will now extract all the data using this dictionary. For this we will make two empty list, one for making nested list with clean android data after removing duplicates and the one to filter out the duplicate data. 

In [8]:
clean_android_data = []
already_added = []

for data in android_data[1:]:
    n_reviews = float(data[3])
    if (reviews_max[data[0]] == n_reviews) and (data[0] not in already_added):
        clean_android_data.append(data)
        already_added.append(data[0])
print(len(clean_android_data))

9659


In order to check if the app is in english (i.e. for english speaking customer) or not, we are going to create a function that tests tha app name based on ascii character. Since, emoji and some characters, such as trademark, can make you omit the app as they are out of ascii range. so, we put a condition that only apps with more than three non-ascii character will be omitted. 

In [9]:
def check_string(a_string):
    ''' this function will check if the string contains english or non-english charactersa'''
    count = 0
    for letter in a_string:
        if ord(letter)>127:
            count += 1
            if count>3:
                return False
        
    return True
print(check_string("Docs To Go™ Free Office Suite"))

True


In [10]:
android_eng_data = []
ios_eng_data = []
for data in clean_android_data:
    if check_string(data[0]):
        android_eng_data.append(data)
for data in ios_data[1:]:
    if check_string(data[1]):
        ios_eng_data.append(data)

Let's check if there was any non-english app in both dataset. Andrpoid dataset has some non-english app whereas ios dataset contains only english app.

In [11]:
print("Inspection of english app for android data: ",end="\n")
print("")
explore_data(clean_android_data,1,2,True)
explore_data(android_eng_data,1,2,True)

Inspection of english app for android data: 

['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows  9659
Number of columns  13
['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows  9614
Number of columns  13


In [12]:
print("Inspection of english app for ios data: ",end="\n")
print("")
explore_data(ios_data[1:],1,2,True)
explore_data(ios_eng_data,1,2,True)

Inspection of english app for ios data: 

['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows  7197
Number of columns  16
['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows  6183
Number of columns  16


Now, we will only select the app that are available for free as we mentioned in the beginning that we are going to make a free app.

In [13]:
free_android_app = []
free_ios_app = []
for data in android_eng_data:
    price = data[7]
    if price == '0':
        free_android_app.append(data)  
for data in ios_eng_data:
    price = data[4]
    if price == '0.0':
        free_ios_app.append(data)

In [14]:
len(free_android_app)
len(free_ios_app)

3222

In [15]:
android_data[:20]

[['App',
  'Category',
  'Rating',
  'Reviews',
  'Size',
  'Installs',
  'Type',
  'Price',
  'Content Rating',
  'Genres',
  'Last Updated',
  'Current Ver',
  'Android Ver'],
 ['Photo Editor & Candy Camera & Grid & ScrapBook',
  'ART_AND_DESIGN',
  '4.1',
  '159',
  '19M',
  '10,000+',
  'Free',
  '0',
  'Everyone',
  'Art & Design',
  'January 7, 2018',
  '1.0.0',
  '4.0.3 and up'],
 ['Coloring book moana',
  'ART_AND_DESIGN',
  '3.9',
  '967',
  '14M',
  '500,000+',
  'Free',
  '0',
  'Everyone',
  'Art & Design;Pretend Play',
  'January 15, 2018',
  '2.0.0',
  '4.0.3 and up'],
 ['U Launcher Lite – FREE Live Cool Themes, Hide Apps',
  'ART_AND_DESIGN',
  '4.7',
  '87510',
  '8.7M',
  '5,000,000+',
  'Free',
  '0',
  'Everyone',
  'Art & Design',
  'August 1, 2018',
  '1.2.4',
  '4.0.3 and up'],
 ['Sketch - Draw & Paint',
  'ART_AND_DESIGN',
  '4.5',
  '215644',
  '25M',
  '50,000,000+',
  'Free',
  '0',
  'Teen',
  'Art & Design',
  'June 8, 2018',
  'Varies with device',
  '4.2 a

So far, we have done the data cleaning task. Now, we will analyse our clean data to see the most popular genre as we want to make the app that will be successful one. 

We are intending to make an app suitable for both android and ios as our final goal is make an app that will be successful in both store. Game genre has been condsider one important sector that has become successful in both app store.

Now, we will createa function to see the most popular genres (category) of apps in both ios and android.

In [16]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    
    for key in table:
        key_val_as_tuple = tuple([table[key], key])
        table_display.append(key_val_as_tuple)
    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [17]:
def freq_table(dataset, index):
    a_column = []
    frequency_table = {}
    for val in dataset:
        data = val[index]
        a_column.append(data)
        
    for data in a_column:
        if data not in frequency_table:
            frequency_table[data] = 1
        else:
            frequency_table[data] += 1
    for key in frequency_table:
        frequency_table[key] =  round((frequency_table[key]/len(dataset)*100),2)
    return frequency_table
    

In [18]:
print("ios app frequency distribution based on Prime Genre: ",end='\n')
display_table(free_ios_app,11)

ios app frequency distribution based on Prime Genre: 
Games : 58.16
Entertainment : 7.88
Photo & Video : 4.97
Education : 3.66
Social Networking : 3.29
Shopping : 2.61
Utilities : 2.51
Sports : 2.14
Music : 2.05
Health & Fitness : 2.02
Productivity : 1.74
Lifestyle : 1.58
News : 1.33
Travel : 1.24
Finance : 1.12
Weather : 0.87
Food & Drink : 0.81
Reference : 0.56
Business : 0.53
Book : 0.43
Navigation : 0.19
Medical : 0.19
Catalogs : 0.12


From the above data, we can conclude that games are most popular in ios followed by Entertainment. Also the data pattern reveals that we should focus on entertainment rather than in building apps for practical purposes.

In [19]:
print("Android app frequency distribution based on Category: ",end='\n')
display_table(free_android_app,1)

Android app frequency distribution based on Category: 
FAMILY : 18.91
GAME : 9.72
TOOLS : 8.46
BUSINESS : 4.59
LIFESTYLE : 3.9
PRODUCTIVITY : 3.89
FINANCE : 3.7
MEDICAL : 3.53
SPORTS : 3.4
PERSONALIZATION : 3.32
COMMUNICATION : 3.24
HEALTH_AND_FITNESS : 3.08
PHOTOGRAPHY : 2.94
NEWS_AND_MAGAZINES : 2.8
SOCIAL : 2.66
TRAVEL_AND_LOCAL : 2.34
SHOPPING : 2.25
BOOKS_AND_REFERENCE : 2.14
DATING : 1.86
VIDEO_PLAYERS : 1.79
MAPS_AND_NAVIGATION : 1.4
FOOD_AND_DRINK : 1.24
EDUCATION : 1.16
ENTERTAINMENT : 0.96
LIBRARIES_AND_DEMO : 0.94
AUTO_AND_VEHICLES : 0.93
HOUSE_AND_HOME : 0.82
WEATHER : 0.8
EVENTS : 0.71
PARENTING : 0.65
ART_AND_DESIGN : 0.64
COMICS : 0.62
BEAUTY : 0.6


Contrary to the ios, Family genre is the most popular one in an googleplay store. 

In [20]:
print("Android app frequency distribution based on Genres: ",end='\n')
display_table(free_android_app,9)

Android app frequency distribution based on Genres: 
Tools : 8.45
Entertainment : 6.07
Education : 5.35
Business : 4.59
Productivity : 3.89
Lifestyle : 3.89
Finance : 3.7
Medical : 3.53
Sports : 3.46
Personalization : 3.32
Communication : 3.24
Action : 3.1
Health & Fitness : 3.08
Photography : 2.94
News & Magazines : 2.8
Social : 2.66
Travel & Local : 2.32
Shopping : 2.25
Books & Reference : 2.14
Simulation : 2.04
Dating : 1.86
Arcade : 1.85
Video Players & Editors : 1.77
Casual : 1.76
Maps & Navigation : 1.4
Food & Drink : 1.24
Puzzle : 1.13
Racing : 0.99
Role Playing : 0.94
Libraries & Demo : 0.94
Auto & Vehicles : 0.93
Strategy : 0.91
House & Home : 0.82
Weather : 0.8
Events : 0.71
Adventure : 0.68
Comics : 0.61
Beauty : 0.6
Art & Design : 0.6
Parenting : 0.5
Card : 0.45
Casino : 0.43
Trivia : 0.42
Educational;Education : 0.39
Board : 0.38
Educational : 0.37
Education;Education : 0.34
Word : 0.26
Casual;Pretend Play : 0.24
Music : 0.2
Racing;Action & Adventure : 0.17
Puzzle;Brain Ga

However,the data varies between category and the genre column. If we observe the data in genre column, we can see tools as the most popular genre. The genre columns seems to be subcategorized further compared to category, so we have to choose based on our requirement, whether we want to have a big piucture or precise genre. In our case, category is better as it reflects the bigger pattern.

If we compare genres between ios and android, we can see they vary a lot. Game genre is most popular in ios but it seems it is sub categorized in android to various genres;sports,actions,action and adventures etc. If we sum-up these genres together asnd consider it as game, it can be consider the most popular and hence can be recommended for both app store. However, on straight comparison with any analysis, Entertainment will be the safe bet. 

########

Isolate the apps of each genre.
Sum up the user ratings for the apps of that genre.
Divide the sum by the number of apps belonging to that genre (not by the total number of apps).

In [21]:
ios_genre = freq_table(free_ios_app,11)

 
for genre in ios_genre:
    total = 0
    len_genre = 0
    for app in free_ios_app:
        genre_app = app[11]
        if genre_app == genre:
            ratings = float(app[5])
            total += ratings
            len_genre += 1
    average_ratings = total/len_genre
    print(genre,':', average_ratings)

Games : 22788.6696905016
Music : 57326.530303030304
Social Networking : 71548.34905660378
Productivity : 21028.410714285714
Business : 7491.117647058823
Medical : 612.0
Education : 7003.983050847458
Entertainment : 14029.830708661417
Travel : 28243.8
Utilities : 18684.456790123455
Weather : 52279.892857142855
Navigation : 86090.33333333333
Reference : 74942.11111111111
Lifestyle : 16485.764705882353
Photo & Video : 28441.54375
News : 21248.023255813954
Catalogs : 4004.0
Book : 39758.5
Health & Fitness : 23298.015384615384
Finance : 31467.944444444445
Food & Drink : 33333.92307692308
Shopping : 26919.690476190477
Sports : 23008.898550724636


In [35]:
android_category = freq_table(free_android_app,1)
for category in android_category:
    total = 0
    len_category = 0
    for app in free_android_app:
        category_app = app[1]
        if category_app == category:
            installs = app[5]
            # since there are + and , sign in this column, we need to remove them
            installs = installs.strip('+')
            installs = installs.replace(',','')
            installs = float(installs)
            total += installs
            len_category += 1
            
    average_installs = total/len_category
    print(category,':', average_installs)

BEAUTY : 513151.88679245283
ENTERTAINMENT : 11640705.88235294
PERSONALIZATION : 5201482.6122448975
HEALTH_AND_FITNESS : 4188821.9853479853
DATING : 854028.8303030303
MEDICAL : 120550.61980830671
LIBRARIES_AND_DEMO : 638503.734939759
EVENTS : 253542.22222222222
FINANCE : 1387692.475609756
SOCIAL : 23253652.127118643
ART_AND_DESIGN : 1986335.0877192982
VIDEO_PLAYERS : 24727872.452830188
TOOLS : 10801391.298666667
PHOTOGRAPHY : 17840110.40229885
SPORTS : 3638640.1428571427
BOOKS_AND_REFERENCE : 8767811.894736841
BUSINESS : 1712290.1474201474
FAMILY : 3695641.8198090694
TRAVEL_AND_LOCAL : 13984077.710144928
COMICS : 817657.2727272727
EDUCATION : 1833495.145631068
SHOPPING : 7036877.311557789
PRODUCTIVITY : 16787331.344927534
WEATHER : 5074486.197183099
MAPS_AND_NAVIGATION : 4056941.7741935486
FOOD_AND_DRINK : 1924897.7363636363
AUTO_AND_VEHICLES : 647317.8170731707
GAME : 15588015.603248259
LIFESTYLE : 1437816.2687861272
NEWS_AND_MAGAZINES : 9549178.467741935
COMMUNICATION : 38456119.16724

### Conclusion letft to be made 